# Домашнее задание к уроку 1. Введение, примеры задач, бизнес- и ML-метрики

## 1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

**Метрики класса Prediction Accuracy — оценивают точность предсказываемого рейтинга:**
- MAE (Mean Absolute Error) - Среднее абсолютное отклонение.
- MSE (Mean Squared Error) - Среднеквадратичная ошибка.
- RMSE (Root Mean Squared Error) - Корень из среднеквадратичной ошибки.

**Метрики класса Decision Support:**
- Precision - доля рекомендаций, понравившихся пользователю.
- Recall - доля интересных пользователю товаров, которая показана.
- F1-Measure - Среднее гармоническое метрик Precision и Recall. Полезно, когда заранее невозможно сказать, какая из метрик важнее.
- ROC AUC - Насколько высока концентрация интересных товаров в начале списка рекомендаций.
- Precision@N - Метрика Precision, посчитанная на Top-N записях.
- Recall@N - Метрика Recall, посчитанная на Top-N записях.
- AverageP - Среднее значение Precision на всем списке рекомендаций.

**Метрики класса Rank Accuracy:**
- Mean Reciprocal Rank - На какой позиции списка рекомендаций пользователь находит первую полезную.
- Spearman Correlation - Корреляция (Спирмена) реального и прогнозируемого рангов рекомендаций.
- nDCG - Информативность выдачи с учетом ранжирования рекомендаций.
- Fraction of Concordance Pairs - Насколько высока концентрация интересных товаров в начале списка рекомендаций.

В качестве метрик также можно использовать ранговые коэффициенты корреляции Кенделла, Мэтьюса, Крамера и другие.

## 2) Доделать все функции, где стоит комментарий "сделать дома"**

In [1]:
import pandas as pd
import numpy as np

In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991] # user1

prices_recommended = [400, 60, 40, 40, 90]
prices_bought = [400, 60, 40, 40 , 90]

Постойте, ведь можно добавить фичи на user (средний чек, кол-во покупок в категориях и т.д), 
на item (цена, кол-во продаж в неделю и т.l), и решать задачу классификации. В чем отличие RecSys от классификации?

 - Много предсказаний для 1 user (extreme classification)
 - Гораздо больший объем данных: 100K users, 10K items --> 1B предсказаний
 - Большинство товаров user никогда не видел --> Не взаимодействовал --> 0 не значит "не понравился"
 - Нет явного таргета. Не понятно, что значит "(не) понравился"
 - Feedback loop
 - Всегда важен порядок рекомендаций

----

# ML-mетрики качества

*Хороши ли рекомендации, предлагаемые моделью?*  
Обычно считаются для каждого юзера, затем усредняются по юзерам

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [3]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

# сделать в домашней работе
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])

    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [4]:
hit_rate(recommended_list, bought_list)

1

In [5]:
hit_rate_at_k(recommended_list, bought_list, k=5)

1

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [6]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

def money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [7]:
precision(recommended_list, bought_list)

0.2

In [8]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [9]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(bought_list)
    
    
    
# сделать дома
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


# сделать дома
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    prices_bought = np.array(prices_bought)

    bought_list = bought_list
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]

    flags = np.isin(recommended_list, bought_list)

    return (prices_recommended * flags).sum() / prices_bought.sum()

In [10]:
recall(recommended_list, bought_list)

0.5

In [11]:
recall_at_k(recommended_list,bought_list,k=10)

0.5

In [12]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.6984126984126984

---

# Метрики ранжирования

Metrics: https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

Если важен порядок рекомендаций.

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка рекомендаций

In [13]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [14]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    for index_relevant in relevant_indexes:
        print(precision_at_k(recommended_list, bought_list, k=index_relevant + 1) )
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    
    return sum_/amount_relevant

In [15]:
ap_k(recommended_list, bought_list, k=5)

1.0
0.5


0.75

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [16]:
# сделать дома
def map_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [17]:
map_k(recommended_list, bought_list, k=5)

0.75

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i+1)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [18]:
# по желанию
def ndcg_at_k(recommended_list, bought_list, k=5):
    recommended_list = recommended_list[:k]
    if recommended_list[0] in bought_list:
        dcg = 1
    else: dcg = 0
    for i in range(1, len(recommended_list)):
        if recommended_list[i] in bought_list:
            dcg += 1/np.log(i+1)
       
    idcg = 1
    for i in range(1, k):
        idcg += 1/np.log(i+1)  
    ndcg = dcg / idcg
    return ndcg

In [19]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.3665860359064237

### Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [20]:
# сделать дома
def reciprocal_rank(recommended_list, bought_list, k=1):
    recommended_list = recommended_list[:k]
    ranks = 0
    for item_rec in recommended_list:
        for i, item_bought in enumerate(bought_list):
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

In [21]:
reciprocal_rank(recommended_list, bought_list, k=5)

0.11666666666666665